In [25]:
from pathlib import Path

import cv2
import pytesseract
import numpy as np
from PIL import Image

video_dir = r'./video/'
pic_dir = r'./picture/'
image_path = Path(f"{pic_dir}/frame.png")
# gray, binary
image = cv2.imread(str(image_path))
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

def show_and_wait(image, *images):
    cv2.imshow("image", image)
    if images:
        for i, image in enumerate(images):
            cv2.imshow(f"image{i}", image)
    k = cv2.waitKey(0)
    if k == ord('s'):
        cv2.destroyAllWindows()

### 순서대로 
#### 1. Edge Detection (가장자리 탐지)
#### 2. Morphological Operations (형태학적 연산)
#### 3. Connected Component Analysis (연결 요소 분석)

In [ ]:
# Edge Detection

edges = cv2.Canny(gray, 100, 200)
show_and_wait(edges)

In [ ]:
# Morphological Transformations

# Define a structuring element
kernel = np.ones((5,5), np.uint8)

# Perform morphological operations
erosion = cv2.erode(image, kernel, iterations = 1)
dilation = cv2.dilate(image, kernel, iterations = 1)
opening = cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)
closing = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)

# Display the results
show_and_wait(image, erosion, dilation, opening, closing)

In [ ]:
# Connected Component Analysis

### tesseract 이용 OCR

In [ ]:
# 이미지 전처리 (예시: 그레이스케일 변환과 이진화)
edges = cv2.Canny(gray, 100, 200)

# 이미지에서 텍스트 추출
text = pytesseract.image_to_string(edges, lang='eng')  # 언어를 적절하게 설정하세요.

# 인식된 텍스트 출력
print(text, len(text))

### 가장 밝은 pixel만 남기기

In [ ]:
image = Image.open(image_path)
pixel_data = image.load()

for y in range(image.size[1]):
    for x in range(image.size[0]):
        r, g, b = pixel_data[x, y]
        if r > 200 and g > 200 and b > 200:
            pixel_data[x, y] = (0, 0, 0)

image.save(f"{pic_dir}/frame2.png")

In [ ]:
image = image.astype(np.float32) / 255.0
threshold = 0.8
white_pixels = np.where(np.sum(image[:, :, :3], axis=2) / 3 > threshold)
print(white_pixels.__len__(), white_pixels[0])
image[white_pixels] = [0, 0, 0, 0]
image = cv2.cvtColor(image, cv2.COLOR_BGR2BGRA)

cv2.imwrite(f"{pic_dir}/frame.png")

In [7]:
# 이진화: Otsu's method를 사용
_, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# Hu Moments를 계산하여 이미지 특성 추출
hu_moments = cv2.HuMoments(cv2.moments(binary)).flatten()

# 결과 출력
print("Hu Moments: ", hu_moments, hu_moments.shape)

Hu Moments:  [1.42444313e-03 5.59433644e-07 9.49004515e-11 9.31733822e-11
 8.69180452e-21 2.24956563e-14 1.10186490e-21] (7,)


In [27]:
# findContours 함수로 윤곽선 검출
contours = binary.copy()
contours, hierachy = cv2.findContours(contours, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(image, contours, -1, (0, 255, 0), 2)
cv2.imshow("contours", image)
cv2.waitKey(1000)
# type(contours)
# len(contours)
# for i in contours:
#     print(i.shape)
# type(image)

-1